# XGBoost y LightGBM

https://cienciadedatos.net/documentos/py54-random-forest-valores-nulos-variable-categoricas

En el ecosistema Python, una de las implementaciones de Random Forest más utilizadas es la disponible en Scikit-learn (RandomForestRegressor, RandomForestClassifier). Aunque esta implementación satisface con éxito la mayoría de los casos de uso, tiene dos limitaciones: no acepta datos ausentes (missing) y carece de la capacidad de manejar de forma nativa variables categóricas. Esto lleva a la necesidad de aplicar estrategias de preprocesamiento (como one hot encoding, target encoding, etc.) para superar estas limitaciones.

XGBoost y LightGBM, conocidos sobre todo por su eficaz aplicación de modelos gradient boosting, también permiten crear modelos Random Forest. Estas implementaciones tienen varias ventajas sobre la de Scikit-learn:

    Manejo nativo de datos ausentes: tienen la capacidad de manejar eficazmente los valores que ausentes. Evitando así la necesidad de eliminarlos o imputarlos.

    Manejo nativo de variables categóricas: A diferencia de la implementación de Scikit-learn, pueden manejar variables categóricas de forma nativa.

    Regularización integrada: incorporan una función de regularización que ayuda a controlar el sobreajuste.

    Velocidad de entrenamiento: gracias a las optimizaciones del algoritmo, pueden entrenar modelos Random Forest más rápido que la implementación Scikit-learn.

    Aceleración GPU: XGBoost y LightGBM tienen una versión compatible con GPU que puede acelerar significativamente el entrenamiento y la inferencia del modelo.


In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Modelado
# ==============================================================================
import xgboost
from xgboost import XGBRFRegressor
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer
import sklearn
import optuna
import time

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
optuna.logging.set_verbosity(optuna.logging.WARNING)

print(f"XGBoost version: {xgboost.__version__}")
print(f"LightGBM version: {lightgbm.__version__}")
print(f"Optuna version: {optuna.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

In [2]:


# Descarga de datos california housing extended
# ==============================================================================
datos = pd.read_csv(
    'https://raw.githubusercontent.com/JoaquinAmatRodrigo/'
    'Estadistica-machine-learning-python/master/data/california_housing_extended.csv'
)
datos = datos.drop(columns=['Latitude', 'Longitude'])
datos['postcode'] = "postcode_" + datos['postcode'].astype(str)
datos['postcode'] = datos['postcode'].replace('postcode_nan', np.nan)
print("Dimensiones de los datos:", datos.shape)
datos.head()



Dimensiones de los datos: (20640, 11)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,county,postcode,ciudad_mas_cercana,distancia_ciudad_mas_cercana
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,4.526,Contra Costa County,postcode_94563.0,Berkeley,3.866682
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,3.585,Contra Costa County,postcode_94563.0,Orinda,4.019485
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,3.521,Alameda County,postcode_94613.0,Piedmont,2.942843
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,3.413,Alameda County,postcode_94613.0,Berkeley,3.122850
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,3.422,Alameda County,postcode_94613.0,Berkeley,3.122850


In [6]:


# Porcentaje de valores nuelos por columna
# ==============================================================================
datos.isnull().mean()



MedInc                          0.000000
HouseAge                        0.000000
AveRooms                        0.000000
AveBedrms                       0.000000
Population                      0.000000
AveOccup                        0.000000
MedHouseVal                     0.000000
county                          0.305281
postcode                        0.102859
ciudad_mas_cercana              0.000000
distancia_ciudad_mas_cercana    0.000000
dtype: float64

In [7]:


# Categorias que aparecen menos de 10 veces se agrupan en una categoria "otro"
# ==============================================================================
limit = 10
for col in ['county', 'ciudad_mas_cercana']:
    counts = datos[col].value_counts()
    under_limit = counts[counts < limit]
    index_under_limit = under_limit.index.tolist()
    datos[col] = datos[col].replace(index_under_limit, 'otro')



In [8]:
# Numero de categorias por variable
# ==============================================================================
print("Numero de categorias por variable")
print("---------------------------------")
print("county:", datos['county'].nunique())
print("postcode:", datos['postcode'].nunique())
print("ciudad_mas_cercana:", datos['ciudad_mas_cercana'].nunique())

Numero de categorias por variable
---------------------------------
county: 55
postcode: 701
ciudad_mas_cercana: 411


In [9]:
# Las variables de tipo categorico se almacenan como dtype category
# ==============================================================================
datos['county'] = datos['county'].astype('category')
datos['postcode'] = datos['postcode'].astype('category')
datos['ciudad_mas_cercana'] = datos['ciudad_mas_cercana'].astype('category')
datos.dtypes

MedInc                           float64
HouseAge                         float64
AveRooms                         float64
AveBedrms                        float64
Population                       float64
AveOccup                         float64
MedHouseVal                      float64
county                          category
postcode                        category
ciudad_mas_cercana              category
distancia_ciudad_mas_cercana     float64
dtype: object

Ojo, si vemos esta parte está partiendo el conjunto de datos en tres: entrenamiento, validación y testeo. Primero hace el train test split usual, aunque yo acostumbro usar test_size pero es lo mismo. De esta manera el 20% de los datos quedan como test y el 80% queda como entrenamiento.

Luego se toman los datos de entrenamiento (80%) y se vuelven a partir en 20/80 donde el 20% que sería de "test" quedan como validación y el 80% restante es para el entrenamiento como tal

In [10]:
# División de los datos en train, validation y test
# ==============================================================================
target = 'MedHouseVal'

X_train, X_test, y_train, y_test = train_test_split(
    datos.drop(columns=target),
    datos[target],
    train_size   = 0.8,
    random_state = 1234,
    shuffle      = True
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    train_size   = 0.8,
    random_state = 1234,
    shuffle      = True
)

print("Observaciones en train:", X_train.shape)
print("Observaciones en validation:", X_val.shape)
print("Observaciones en test:", X_test.shape)

Observaciones en train: (13209, 10)
Observaciones en validation: (3303, 10)
Observaciones en test: (4128, 10)


## Comparación de modelos

A continuación, se compararán los modelos Random Forest de las librerías scikit-learn, XGBoost y LightGBM en términos de velocidad de entrenamiento, velocidad de inferencia y capacidad predictiva.

Para todos los modelos, se realiza una **búsqueda de hiperparámetros utilizado la librería Optuna**. La selección de hiperparámetros se **realiza con los datos de validación**. En el caso de scikit-learn, dado que no tiene una implementación nativa para manejar variables categóricas, se utiliza un preprocesamiento con OneHotEncoder para codificar las variables categóricas.


In [11]:


# Target encoding de las variables categóricas
# ==============================================================================
col_categoricas = datos.select_dtypes(exclude=np.number).columns.to_list()
encoder = ColumnTransformer(
    transformers=[
        ('target', TargetEncoder(target_type="continuous"), col_categoricas)
    ],
    remainder='passthrough'
).set_output(transform='pandas')

encoder.fit(X_train, y_train)
X_train_encoded = encoder.transform(X_train)
X_val_encoded   = encoder.transform(X_val)
X_test_encoded  = encoder.transform(X_test)

print("Observaciones en train encoded:", X_train_encoded.shape)
print("Observaciones en validation encoded:", X_val_encoded.shape)
print("Observaciones en test encoded:", X_test_encoded.shape)



Observaciones en train encoded: (13209, 10)
Observaciones en validation encoded: (3303, 10)
Observaciones en test encoded: (4128, 10)


In [12]:
X_train_encoded

,target__county,target__postcode,target__ciudad_mas_cercana,remainder__MedInc,remainder__HouseAge,remainder__AveRooms,remainder__AveBedrms,remainder__Population,remainder__AveOccup,remainder__distancia_ciudad_mas_cercana
5682,2.470523,2.980141,3.300979,3.2740,39.0,5.218504,1.161417,1103.0,2.171260,7.202112
3925,2.470523,2.673632,2.643932,4.8264,36.0,5.059259,0.829630,719.0,2.662963,8.771031
13903,1.251811,0.829294,0.790397,2.0674,14.0,5.225367,1.297694,2517.0,2.638365,28.828879
2674,0.726556,0.935648,0.604444,3.3472,31.0,5.607143,1.089286,152.0,2.714286,2.016254
16361,1.190123,0.960024,0.980530,3.2788,29.0,5.262069,1.032184,1215.0,2.793103,5.871032
...,...,...,...,...,...,...,...,...,...,...
6210,2.470523,1.956860,2.271391,3.0550,35.0,5.878571,1.050000,1056.0,3.771429,2.396115
9275,3.494280,2.594526,2.752459,3.4705,32.0,4.869986,1.034578,1706.0,2.359613,0.288168
17203,2.727429,3.167183,3.367475,4.5278,35.0,4.283677,1.036450,1383.0,2.191759,3.090729
12735,1.320777,1.533666,1.584132,5.9174,23.0,7.102439,1.026829,1132.0,2.760976,3.241098


In [13]:


# Búsqueda bayesiana de hiperparámetros con optuna
# ==============================================================================
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'max_features': trial.suggest_float('max_features', 0.3, 1),
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 1),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 1),        
    }

    model = RandomForestRegressor(
                n_jobs       = -1,
                random_state = 4576688,
                criterion    = 'squared_error',
                **params
            )

# Acá podemos ver que la predicción se hace con los datos de validación y no con los datos de test    
    
    model.fit(X_train_encoded, y_train)
    predictions = model.predict(X_val_encoded)
    score = mean_squared_error(y_val, predictions, squared=False)
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25, show_progress_bar=True, timeout=60*5)

print('Mejores hiperparámetros:', study.best_params)
print('Mejor score:', study.best_value)



  0%|          | 0/25 [00:00<?, ?it/s]

Mejores hiperparámetros: {'n_estimators': 900, 'max_depth': 13, 'min_samples_split': 69, 'min_samples_leaf': 76, 'max_features': 0.5979907311013706, 'ccp_alpha': 0.11409341585279496, 'min_impurity_decrease': 0.0012502289026489755}
Mejor score: 0.7246189846044433


In [14]:


# Random Forest scikit-learn con los mejores hiperparámetros encontrados
# ==============================================================================
rf_sklearn = RandomForestRegressor(
                n_jobs       = -1,
                random_state = 4576688,
                criterion    = 'squared_error',
                **study.best_params
            )

# Entrenamiento del modelo
start = time.time()
rf_sklearn.fit(
        X = pd.concat([X_train_encoded, X_val_encoded]),
        y = pd.concat([y_train, y_val])
)
end = time.time()
tiempo_entrenamiento_sklearn = end - start

# Predicciones test
start = time.time()
predicciones = rf_sklearn.predict(X=X_test_encoded)
end = time.time()
tiempo_prediccion_sklearn = end - start

# Error de test del modelo
rmse_rf_sklearn = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )

print(f"Tiempo entrenamiento: {tiempo_entrenamiento_sklearn:.2f} segundos")
print(f"Tiempo predicción:    {tiempo_prediccion_sklearn:.2f} segundos")
print(f"RMSE:                 {rmse_rf_sklearn:.2f}")



Tiempo entrenamiento: 4.15 segundos
Tiempo predicción:    0.09 segundos
RMSE:                 0.75


# Random Forest con XGBoost

In [15]:


# Búsqueda bayesiana de hiperparámetros con optuna
# ==============================================================================
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1e+3, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1e+3, log=True),
        'gamma': trial.suggest_float('gamma', 1e-5, 1e+3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1),
    }

    model = XGBRFRegressor(
                    tree_method        = 'hist',
                    grow_policy        = 'depthwise',
                    learning_rate      = 1.0,
                    n_jobs             = -1,
                    random_state       = 4576,
                    enable_categorical = True,
                    **params
            )
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    score = mean_squared_error(y_val, predictions, squared=False)
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25, show_progress_bar=True, timeout=60*10)

print('Mejores hiperparámetros:', study.best_params)
print('Mejor score:', study.best_value)



  0%|          | 0/25 [00:00<?, ?it/s]

Mejores hiperparámetros: {'n_estimators': 600, 'max_depth': 27, 'reg_lambda': 0.0029925737638358835, 'reg_alpha': 2.5808608983578098e-05, 'gamma': 0.01739933211623556, 'subsample': 0.6714497154246463, 'colsample_bynode': 0.5901611888453542}
Mejor score: 0.43700089681200505


In [16]:
# Random Forest XGBoost con los mejores hiperparámetros encontrados
# ==============================================================================
rf_xgboost = XGBRFRegressor(
                tree_method        = 'hist',
                grow_policy        = 'depthwise',
                learning_rate      = 1.0,
                n_jobs             = -1,
                random_state       = 4576,
                enable_categorical = True,
                **study.best_params
             )

# Entrenamiento del modelo
start = time.time()
rf_xgboost.fit(
        X = pd.concat([X_train, X_val]),
        y = pd.concat([y_train, y_val])
)
end = time.time()
tiempo_entrenamiento_xgboost = end - start

# Predicciones test
start = time.time()
predicciones = rf_xgboost.predict(X=X_test)
end = time.time()
tiempo_prediccion_xgboost = end - start

# Error de test del modelo
rmse_rf_xgboost = mean_squared_error(
                      y_true  = y_test,
                      y_pred  = predicciones,
                      squared = False
                  )
                 
print(f"Tiempo entrenamiento: {tiempo_entrenamiento_xgboost:.2f} segundos")
print(f"Tiempo predicción:    {tiempo_prediccion_xgboost:.2f} segundos")
print(f"RMSE:                 {rmse_rf_xgboost:.2f}")

Tiempo entrenamiento: 98.78 segundos
Tiempo predicción:    0.29 segundos
RMSE:                 0.46


# Random Forest con LightGBM

In [17]:
# Búsqueda bayesiana de hiperparámetros con optuna
# ==============================================================================
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'num_leaves': trial.suggest_int('num_leaves', 5, 256),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1e+3, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1e+3, log=True),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.3, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
    }

    model = LGBMRegressor(
                boosting_type  = 'rf',
                learning_rate  = 1.0,
                subsample_freq = 1,
                n_jobs         = -1,
                random_state   = 4576688,
                verbose        = -1,
                **params
            )
    model.fit(X_train, y_train, categorical_feature='auto')
    predictions = model.predict(X_val)
    score = mean_squared_error(y_val, predictions, squared=False)
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25, show_progress_bar=True, timeout=60*5)

print('Mejores hiperparámetros:', study.best_params)
print('Mejor score:', study.best_value)

  0%|          | 0/25 [00:00<?, ?it/s]

Mejores hiperparámetros: {'n_estimators': 600, 'num_leaves': 100, 'reg_lambda': 0.000244770264280912, 'reg_alpha': 2.3708704495820483e-05, 'min_samples_leaf': 27, 'colsample_bytree': 0.9098641805630351, 'colsample_bynode': 0.42720012924251666, 'subsample': 0.8267850571507624}
Mejor score: 0.4857546614551951


In [18]:
# Random Forest LightGBM con los mejores hiperparámetros encontrados
# ==============================================================================
rf_lgbm = LGBMRegressor(
                boosting_type  = 'rf',
                learning_rate  = 1.0,
                subsample_freq = 1,
                n_jobs         = -1,
                random_state   = 4576688,
                verbose        = -1,
                **study.best_params
          )

# Entrenamiento del modelo
start = time.time()
rf_lgbm.fit(X_train, y_train, categorical_feature='auto')
end = time.time()
tiempo_entrenamiento_lgbm = end - start

# Predicciones test
start = time.time()
predicciones = rf_lgbm.predict(X=X_test)
end = time.time()
tiempo_prediccion_lgbm = end - start

# Error de test del modelo
rmse_rf_lgbm = mean_squared_error(
                y_true  = y_test,
                y_pred  = predicciones,
                squared = False
               )

print(f"Tiempo entrenamiento: {tiempo_entrenamiento_lgbm:.2f} segundos")
print(f"Tiempo predicción:    {tiempo_prediccion_lgbm:.2f} segundos")
print(f"RMSE:                 {rmse_rf_lgbm:.2f}")

Tiempo entrenamiento: 1.09 segundos
Tiempo predicción:    0.05 segundos
RMSE:                 0.52


# Comparativa de modelos

In [19]:

# Comparativa de modelos
# ==============================================================================
resultados = pd.DataFrame({
               'rmse': [rmse_rf_sklearn, rmse_rf_xgboost, rmse_rf_lgbm],
               'tiempo_entrenamiento': [
                  tiempo_entrenamiento_sklearn,
                  tiempo_entrenamiento_xgboost,
                  tiempo_entrenamiento_lgbm
                ],
               'tiempo_prediccion': [
                  tiempo_prediccion_sklearn,
                  tiempo_prediccion_xgboost,
                  tiempo_prediccion_lgbm
                ]
               },
               index = [
                  'Random Forest sklearn',
                  'Random Forest XGBoost',
                  'Random Forest LightGBM'
                ]
             )
resultados.style.highlight_min(axis=0, color='green').format(precision=2)



,rmse,tiempo_entrenamiento,tiempo_prediccion
Random Forest sklearn,0.75,4.15,0.09
Random Forest XGBoost,0.46,98.78,0.29
Random Forest LightGBM,0.52,1.09,0.05
